In [ ]:
import requests
from datetime import datetime
from urllib.parse import quote
import time

BASE_URL = "http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno"

def get_partenze(station_code, dt):
    # Viaggiatreno expects a string like:
    # "Wed Mar 08 2023 17:04:00 GMT+0100" EN format.
    date_str = dt.strftime("%a %b %d %Y %H:%M:%S GMT+0100")

    # String URL encode
    date_encoded = quote(date_str, safe="")

    url = f"{BASE_URL}/partenze/{station_code}/{date_encoded}"
    resp = requests.get(url, timeout=10)

    print(f"\n=== Stazione {station_code} ===")
    print("URL:", url)
    print("Status code:", resp.status_code)
    print("Content-Type:", resp.headers.get("Content-Type"))
    print("Latenza:", resp.elapsed.total_seconds(), "s")

    body = resp.text
    print("Primi 300 caratteri di risposta:")
    print(body[:300])

    return resp.status_code, resp.elapsed.total_seconds(), body


stations_test = ["S01700", "S01645"]  # Centrale, Garibaldi
results = []
for st in stations_test:
    status, latency, body = get_partenze(st, datetime.now())
    results.append({
        "station": st,
        "status": status,
        "latency": latency,
        "len_body": len(body)
    })

    with open(f"sample_partenze_{st}.json", "w", encoding="utf-8") as f:
        f.write(body)

    time.sleep(1.0)



=== Stazione S01700 ===
URL: http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/partenze/S01700/Sat%20Dec%2006%202025%2018%3A00%3A00%20GMT%2B0100
Status code: 200
Content-Type: application/json
Latenza: 0.716168 s
Primi 300 caratteri di risposta:
[{"arrivato":true,"dataPartenzaTrenoAsDate":"2025-12-06","dataPartenzaTreno":1764975600000,"partenzaTreno":1765043190000,"millisDataPartenza":"1764975600000","numeroTreno":25530,"categoria":"REG","categoriaDescrizione":"REG","origine":null,"codOrigine":"S01700","destinazione":"CHIASSO","codDestinazi

=== Stazione S01645 ===
URL: http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/partenze/S01645/Sat%20Dec%2006%202025%2018%3A00%3A01%20GMT%2B0100
Status code: 200
Content-Type: application/json
Latenza: 0.577978 s
Primi 300 caratteri di risposta:
[{"arrivato":true,"dataPartenzaTrenoAsDate":"2025-12-06","dataPartenzaTreno":1764975600000,"partenzaTreno":1765043310000,"millisDataPartenza":"1764975600000","numeroTreno":25068,"ca

In [ ]:
import json

data = json.loads(body)
print(len(data))
print(data[0])

12
{'arrivato': True, 'dataPartenzaTrenoAsDate': '2025-11-29', 'dataPartenzaTreno': 1764370800000, 'partenzaTreno': 1764430860000, 'millisDataPartenza': '1764370800000', 'numeroTreno': 25060, 'categoria': 'REG', 'categoriaDescrizione': 'REG', 'origine': None, 'codOrigine': 'S01645', 'destinazione': 'CHIASSO', 'codDestinazione': None, 'origineEstera': None, 'destinazioneEstera': None, 'oraPartenzaEstera': None, 'oraArrivoEstera': None, 'tratta': 0, 'regione': 0, 'origineZero': None, 'destinazioneZero': None, 'orarioPartenzaZero': None, 'orarioArrivoZero': None, 'circolante': True, 'codiceCliente': 63, 'binarioEffettivoArrivoCodice': None, 'binarioEffettivoArrivoDescrizione': None, 'binarioEffettivoArrivoTipo': None, 'binarioProgrammatoArrivoCodice': None, 'binarioProgrammatoArrivoDescrizione': None, 'binarioEffettivoPartenzaCodice': '17', 'binarioEffettivoPartenzaDescrizione': '15', 'binarioEffettivoPartenzaTipo': '0', 'binarioProgrammatoPartenzaCodice': None, 'binarioProgrammatoPartenz

In [ ]:
print(data[0].keys())

dict_keys(['arrivato', 'dataPartenzaTrenoAsDate', 'dataPartenzaTreno', 'partenzaTreno', 'millisDataPartenza', 'numeroTreno', 'categoria', 'categoriaDescrizione', 'origine', 'codOrigine', 'destinazione', 'codDestinazione', 'origineEstera', 'destinazioneEstera', 'oraPartenzaEstera', 'oraArrivoEstera', 'tratta', 'regione', 'origineZero', 'destinazioneZero', 'orarioPartenzaZero', 'orarioArrivoZero', 'circolante', 'codiceCliente', 'binarioEffettivoArrivoCodice', 'binarioEffettivoArrivoDescrizione', 'binarioEffettivoArrivoTipo', 'binarioProgrammatoArrivoCodice', 'binarioProgrammatoArrivoDescrizione', 'binarioEffettivoPartenzaCodice', 'binarioEffettivoPartenzaDescrizione', 'binarioEffettivoPartenzaTipo', 'binarioProgrammatoPartenzaCodice', 'binarioProgrammatoPartenzaDescrizione', 'subTitle', 'esisteCorsaZero', 'orientamento', 'inStazione', 'haCambiNumero', 'nonPartito', 'provvedimento', 'riprogrammazione', 'orarioPartenza', 'orarioArrivo', 'stazionePartenza', 'stazioneArrivo', 'statoTreno', '

In [ ]:
from datetime import datetime, timezone
import json

def ms_to_dt(ms):
    if ms is None:
        return None
    return datetime.fromtimestamp(ms / 1000, tz=timezone.utc).astimezone()

def normalize_train_record(rec: dict, station_code: str) -> dict:
    # fallback: i saw that a lot of partenzaTreno are None (around 70%),
    # so we can use orarioPartenza, that is always valorized
    sched_ms = rec.get("partenzaTreno") or rec.get("orarioPartenza")
    real_ms = rec.get("orarioPartenza")  # we use this as "real"

    ritardo_raw = rec.get("ritardo")

    # ritardo_clean: -1 -> None
    if ritardo_raw == -1:
        ritardo_clean = None
    else:
        ritardo_clean = ritardo_raw

    return {
        "train_number": rec.get("numeroTreno"),
        "data_operativa": rec.get("dataPartenzaTrenoAsDate"),
        "id_stazione": station_code,
        "orario_sched": ms_to_dt(sched_ms),
        "orario_reale": ms_to_dt(real_ms),
        "ritardo_raw": ritardo_raw,
        "ritardo_clean": ritardo_clean,
        "json_raw": json.dumps(rec, ensure_ascii=False),
    }


In [ ]:
rows = []
station_code = "S01700"  # Centrale

for rec in data:
    rows.append(normalize_train_record(rec, station_code))

print(rows[0])

{'train_number': 2962, 'data_operativa': '2025-11-29', 'id_stazione': 'S01700', 'orario_sched': datetime.datetime(2025, 11, 29, 15, 55, 30, tzinfo=datetime.timezone(datetime.timedelta(0), 'UTC')), 'orario_reale': datetime.datetime(2025, 11, 29, 16, 5, tzinfo=datetime.timezone(datetime.timedelta(0), 'UTC')), 'ritardo_raw': 2, 'ritardo_clean': 2, 'json_raw': '{"arrivato": true, "dataPartenzaTrenoAsDate": "2025-11-29", "dataPartenzaTreno": 1764370800000, "partenzaTreno": 1764431730000, "millisDataPartenza": "1764370800000", "numeroTreno": 2962, "categoria": "REG", "categoriaDescrizione": "REG", "origine": null, "codOrigine": "S01700", "destinazione": "MALPENSA AEROPORTO TERMINAL 2", "codDestinazione": null, "origineEstera": null, "destinazioneEstera": null, "oraPartenzaEstera": null, "oraArrivoEstera": null, "tratta": 0, "regione": 0, "origineZero": null, "destinazioneZero": null, "orarioPartenzaZero": null, "orarioArrivoZero": null, "circolante": true, "codiceCliente": 63, "binarioEffett

In [ ]:
all_rows = []

for st in stations_test:
    status, latency, body = get_partenze(st, datetime.now())
    print(f"{st} → status {status}, latency {latency:.3f}s")

    if status != 200 or not body.strip():
        print(f"  Nessun dato valido per {st}")
        continue

    try:
        data = json.loads(body)
    except json.JSONDecodeError as e:
        print(f"  ERRORE JSON per {st}:", e)
        continue

    print(f"  Record trovati: {len(data)}")

    for rec in data:
        all_rows.append(normalize_train_record(rec, st))

    time.sleep(1.0)  # throttle molto conservativo

print("\nTotale righe normalizzate:", len(all_rows))
if all_rows:
    print("Esempio riga:")
    print(all_rows[0])


=== Stazione S01700 ===
URL: http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/partenze/S01700/Sat%20Nov%2029%202025%2016%3A29%3A46%20GMT%2B0100
Status code: 200
Content-Type: application/json
Latenza: 0.800973 s
Primi 300 caratteri di risposta:
[{"arrivato":true,"dataPartenzaTrenoAsDate":"2025-11-29","dataPartenzaTreno":1764370800000,"partenzaTreno":1764432930000,"millisDataPartenza":"1764370800000","numeroTreno":2175,"categoria":"REG","categoriaDescrizione":"REG","origine":null,"codOrigine":"S01700","destinazione":"BOZZOLO","codDestinazio
S01700 → status 200, latency 0.801s
  Record trovati: 16

=== Stazione S01645 ===
URL: http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/partenze/S01645/Sat%20Nov%2029%202025%2016%3A29%3A48%20GMT%2B0100
Status code: 200
Content-Type: application/json
Latenza: 0.725536 s
Primi 300 caratteri di risposta:
[{"arrivato":true,"dataPartenzaTrenoAsDate":"2025-11-29","dataPartenzaTreno":1764370800000,"partenzaTreno":1764432900000,"mi

Little quality check on the records we retrieved, containing:
- n of records
- how many got essential columns missing
- how many has negative delays - to treat as null i guess


In [ ]:
def quality_summary(rows):
    n = len(rows)
    if n == 0:
        print("Nessuna riga da analizzare.")
        return

    missing_train_number = sum(1 for r in rows if r["train_number"] is None)
    missing_sched = sum(1 for r in rows if r["orario_sched"] is None)
    missing_real = sum(1 for r in rows if r["orario_reale"] is None)
    ritardo_minus1 = sum(1 for r in rows if r["ritardo_raw"] == -1)
    ritardo_new = sum(1 for r in rows if r["ritardo_clean"] == -1)

    print(f"Totale righe: {n}")
    print(f"- train_number missing: {missing_train_number} ({missing_train_number/n:.1%})")
    print(f"- orario_sched missing: {missing_sched} ({missing_sched/n:.1%})")
    print(f"- orario_reale missing: {missing_real} ({missing_real/n:.1%})")
    print(f"- ritardo = -1: {ritardo_minus1} ({ritardo_minus1/n:.1%})")
    print(f"- ritardo = -1 fixato: {ritardo_new} ({ritardo_new/n:.1%})")

quality_summary(all_rows)


Totale righe: 29
- train_number missing: 0 (0.0%)
- orario_sched missing: 0 (0.0%)
- orario_reale missing: 0 (0.0%)
- ritardo = -1: 2 (6.9%)
- ritardo = -1 fixato: 0 (0.0%)


I think the data is usable, didn't do much quality processing but the /partenze (departures) endpoint works, the JSON is clear, some missing values and some quality problems but we can go through i think


**Rate limits**

I think we should work on the Milan/S trains (we can discuss it) but i think is a good balance.

We can do a division based on the affluence: on the big hubs (Centrale, Garibaldi, Rogoredo...) 1 call every 5 mins, on suburbans ones (Bovisa, sesto, greco Pirelli...) 1 call every 15 mins

So around 64 req/hour that is not much, i don't know already if we have to use the /andamentotreno endpoint tho

In [ ]:
def throughput_probe(stations, delay_sec=0.8, cycles=3):
    """
    Performs a small throughput test:
    - stations: list of station codes
    - delay_sec: pause between ONE call and the next
    - cycles: how many times to repeat the cycle on the stations
    Returns a list of logs with info on status and latency.
    """
    logs = []
    for c in range(cycles):
        print(f"\n=== Ciclo {c+1}/{cycles} ===")
        for st in stations:
            status, latency, body = get_partenze(st, datetime.now())
            print(f"{datetime.now().isoformat()}  {st} → {status} ({latency:.3f}s) len={len(body)}")

            logs.append({
                "timestamp": datetime.now().isoformat(),
                "station": st,
                "status": status,
                "latency": latency,
                "len_body": len(body)
            })

            time.sleep(delay_sec)
    return logs

In [ ]:
logs = throughput_probe(stations_test, delay_sec=0.8, cycles=5)
print("\nTotal requests:", len(logs))


=== Ciclo 1/5 ===

=== Stazione S01700 ===
URL: http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/partenze/S01700/Sat%20Nov%2029%202025%2016%3A44%3A18%20GMT%2B0100
Status code: 200
Content-Type: application/json
Latenza: 0.695559 s
Primi 300 caratteri di risposta:
[{"arrivato":false,"dataPartenzaTrenoAsDate":"2025-11-29","dataPartenzaTreno":1764370800000,"partenzaTreno":null,"millisDataPartenza":"1764370800000","numeroTreno":9811,"categoria":"","categoriaDescrizione":" FR","origine":null,"codOrigine":"S01700","destinazione":"PESCARA","codDestinazione":null,"o
2025-11-29T16:44:18.735330  S01700 → 200 (0.696s) len=41610

=== Stazione S01645 ===
URL: http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/partenze/S01645/Sat%20Nov%2029%202025%2016%3A44%3A19%20GMT%2B0100
Status code: 200
Content-Type: application/json
Latenza: 0.63417 s
Primi 300 caratteri di risposta:
[{"arrivato":true,"dataPartenzaTrenoAsDate":"2025-11-29","dataPartenzaTreno":1764370800000,"partenzaTre

In [ ]:
from collections import Counter

statuses = Counter(log["status"] for log in logs)
print("Status code osservati:")
for s, count in statuses.items():
    print(f"- {s}: {count}")

if logs:
    avg_latency = sum(l["latency"] for l in logs) / len(logs)
    print(f"\nLatenza media: {avg_latency:.3f} s")


Status code osservati:
- 200: 10

Latenza media: 0.671 s


In the test with 10 requests distributed across 2 stations with a 0.8-second delay between requests, all status codes were 200, with no evidence of server-side rate limiting.